In [1]:
!pip install opendatasets

In [2]:
# importing all the necessary libraries
import opendatasets as od
import os
import cv2
import numpy as np
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [3]:
od.download("https://www.kaggle.com/datasets/arunrk7/surface-crack-detection")  # loading the dataset

Skipping, found downloaded files in "./surface-crack-detection" (use force=True to force download)


Data/image preprocessing

Collecting all the images and label in variables seperately

In [4]:
negativeimages = os.listdir("/content/surface-crack-detection/Negative/")
negativeimages = negativeimages[:5000]
images = []
labels = []
negative_images_folder_path = "/content/surface-crack-detection/Negative/"
for image in negativeimages:
  img_path = os.path.join(negative_images_folder_path, image)
  img = cv2.imread(img_path)
  img = cv2.resize(img, (75,75))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  images.append(img)
  labels.append(0)   # 0 for negative(not cracked images)

In [5]:
positiveimages = os.listdir("/content/surface-crack-detection/Positive/")
positiveimages = positiveimages[:5000]
positive_images_folder_path = "/content/surface-crack-detection/Positive/"
for image in positiveimages:
  img_path = os.path.join(positive_images_folder_path, image)
  img = cv2.imread(img_path)
  img = cv2.resize(img, (75,75))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  images.append(img)
  labels.append(1)   # 1 for positive(cracked images)

print(f'No of images -', len(images))
print(f'No of labels -', len(labels))

No of images - 10000
No of labels - 10000


In [6]:
set(labels)

{0, 1}

In [7]:
images = np.array(images)
labels = np.array(labels)

In [8]:
xtrain,xtest = images[3000:7000],images[4500:5500]   # getting 4000 images for training and 1000 for testing
ytrain,ytest = labels[3000:7000],labels[4500:5500]

print(f'xtrain shape is -', xtrain.shape)
print(f'xtest shape is -', xtest.shape)

xtrain shape is - (4000, 75, 75, 3)
xtest shape is - (1000, 75, 75, 3)


In [9]:
xtrain

array([[[[196, 192, 181],
         [196, 192, 181],
         [197, 193, 182],
         ...,
         [199, 196, 189],
         [199, 196, 189],
         [202, 199, 192]],

        [[196, 192, 181],
         [197, 193, 182],
         [199, 195, 184],
         ...,
         [197, 194, 187],
         [201, 198, 191],
         [201, 198, 191]],

        [[201, 197, 186],
         [197, 193, 182],
         [195, 191, 180],
         ...,
         [197, 194, 187],
         [199, 196, 189],
         [203, 200, 193]],

        ...,

        [[198, 195, 188],
         [195, 192, 185],
         [194, 191, 184],
         ...,
         [202, 197, 191],
         [204, 199, 193],
         [206, 201, 195]],

        [[197, 194, 187],
         [198, 195, 188],
         [195, 192, 185],
         ...,
         [201, 196, 190],
         [210, 205, 199],
         [208, 203, 197]],

        [[198, 195, 188],
         [197, 194, 187],
         [196, 193, 186],
         ...,
         [203, 198, 192],
        

In [10]:
xtrain = preprocess_input(xtrain)
xtest = preprocess_input(xtest)

In [11]:
xtrain

array([[[[ 0.5372549 ,  0.5058824 ,  0.41960788],
         [ 0.5372549 ,  0.5058824 ,  0.41960788],
         [ 0.54509807,  0.5137255 ,  0.427451  ],
         ...,
         [ 0.56078434,  0.5372549 ,  0.48235297],
         [ 0.56078434,  0.5372549 ,  0.48235297],
         [ 0.58431375,  0.56078434,  0.5058824 ]],

        [[ 0.5372549 ,  0.5058824 ,  0.41960788],
         [ 0.54509807,  0.5137255 ,  0.427451  ],
         [ 0.56078434,  0.5294118 ,  0.4431373 ],
         ...,
         [ 0.54509807,  0.52156866,  0.4666667 ],
         [ 0.5764706 ,  0.5529412 ,  0.49803925],
         [ 0.5764706 ,  0.5529412 ,  0.49803925]],

        [[ 0.5764706 ,  0.54509807,  0.45882356],
         [ 0.54509807,  0.5137255 ,  0.427451  ],
         [ 0.5294118 ,  0.49803925,  0.41176474],
         ...,
         [ 0.54509807,  0.52156866,  0.4666667 ],
         [ 0.56078434,  0.5372549 ,  0.48235297],
         [ 0.5921569 ,  0.5686275 ,  0.5137255 ]],

        ...,

        [[ 0.5529412 ,  0.5294118 ,  0

Training the InceptionV3 model

In [12]:
inceptionV3model = InceptionV3(weights = "imagenet",include_top = False,input_shape = (75,75,3))  # InceptionV3 block

cnn_block = inceptionV3model.output  # CNN block

flattenpart = GlobalAveragePooling2D()(cnn_block)  # Flatten block

# ANN block
annpart = Dense(1924,activation = "relu")(flattenpart)
hiddenann = Dense(520,activation = "relu")(annpart)
outputlayer = Dense(1,activation = "sigmoid")(hiddenann)
model = Model(inputs = inceptionV3model.input, outputs = outputlayer)

# Compiling the model
model.compile(optimizer = "adam", loss = "binary_crossentropy",metrics = ["accuracy"])
model.fit(xtrain,ytrain, epochs = 3 , batch_size = 32 ,validation_data = (xtest,ytest))

Epoch 1/3
125/125 [==============================] - 49s 80ms/step - loss: 0.2111 - accuracy: 0.9555 - val_loss: 0.2333 - val_accuracy: 0.9930
Epoch 2/3
125/125 [==============================] - 7s 54ms/step - loss: 0.0336 - accuracy: 0.9923 - val_loss: 0.0818 - val_accuracy: 0.9800
Epoch 3/3
125/125 [==============================] - 7s 59ms/step - loss: 0.0207 - accuracy: 0.9942 - val_loss: 0.0095 - val_accuracy: 0.9960


In [13]:
model.evaluate(xtest,ytest)

32/32 [==============================] - 1s 18ms/step - loss: 0.0095 - accuracy: 0.9960


[0.009467411786317825, 0.9959999918937683]

Checking prediction of model on cracked image

In [14]:
img_path = "/content/surface-crack-detection/Positive/00004.jpg"
image = cv2.imread(img_path)
image = cv2.resize(image,(75,75))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis = 0)
image = image / 255.0
prediction = model.predict(image)
if prediction[0][0] >= 0.5:
  print("1 => positive => crack detected")
else:
  print("0 => negative => crack not detected")


1/1 [==============================] - 2s 2s/step
1 => positive => crack detected


Checking prediction of model on not-cracked image

In [15]:
img_path = "/content/surface-crack-detection/Negative/00003.jpg"
image = cv2.imread(img_path)
image = cv2.resize(image,(75,75))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis = 0)
image = image / 255.0
prediction = model.predict(image)
if prediction[0][0] >= 0.5:
  print("1 => positive => crack detected")
else:
  print("0 => negative => crack not detected")


1/1 [==============================] - 0s 27ms/step
0 => negative => crack not detected
